In [0]:
help(spark.createDataFrame)

Help on method createDataFrame in module pyspark.sql.session:

createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True) method of pyspark.sql.session.SparkSession instance
 Creates a :class:`DataFrame` from an :class:`RDD`, a list or a :class:`pandas.DataFrame`.
 
 When ``schema`` is a list of column names, the type of each column
 will be inferred from ``data``.
 
 When ``schema`` is ``None``, it will try to infer the schema (column names and types)
 from ``data``, which should be an RDD of either :class:`Row`,
 :class:`namedtuple`, or :class:`dict`.
 
 When ``schema`` is :class:`pyspark.sql.types.DataType` or a datatype string, it must match
 the real data, or an exception will be thrown at runtime. If the given schema is not
 :class:`pyspark.sql.types.StructType`, it will be wrapped into a
 :class:`pyspark.sql.types.StructType` as its only field, and the field name will be "value".
 Each record will also be wrapped into a tuple, which can be converted to row later.
 
 If schema inference is needed, ``samplingRatio`` is used to determined the ratio of
 rows used for schema inference. The first row will be used if ``samplingRatio`` is ``None``.
 
 .. versionadded:: 2.0.0
 
 .. versionchanged:: 2.1.0
 Added verifySchema.
 
 Parameters
 ----------
 data : :class:`RDD` or iterable
 an RDD of any kind of SQL data representation (:class:`Row`,
 :class:`tuple`, ``int``, ``boolean``, etc.), or :class:`list`, or
 :class:`pandas.DataFrame`.
 schema : :class:`pyspark.sql.types.DataType`, str or list, optional
 a :class:`pyspark.sql.types.DataType` or a datatype string or a list of
 column names, default is None. The data type string format equals to
 :class:`pyspark.sql.types.DataType.simpleString`, except that top level struct type can
 omit the ``struct<>`` and atomic types use ``typeName()`` as their format, e.g. use
 ``byte`` instead of ``tinyint`` for :class:`pyspark.sql.types.ByteType`.
 We can also use ``int`` as a short name for :class:`pyspark.sql.types.IntegerType`.
 samplingRatio : float, optional
 the sample ratio of rows used for inferring
 verifySchema : bool, optional
 verify data types of every row against schema. Enabled by default.
 
 Returns
 -------
 :class:`DataFrame`
 
 Notes
 -----
 Usage with spark.sql.execution.arrow.pyspark.enabled=True is experimental.
 
 Examples
 --------
 >>> l = [('Alice', 1)]
 >>> spark.createDataFrame(l).collect()
 [Row(_1='Alice', _2=1)]
 >>> spark.createDataFrame(l, ['name', 'age']).collect()
 [Row(name='Alice', age=1)]
 
 >>> d = [{'name': 'Alice', 'age': 1}]
 >>> spark.createDataFrame(d).collect()
 [Row(age=1, name='Alice')]
 
 >>> rdd = sc.parallelize(l)
 >>> spark.createDataFrame(rdd).collect()
 [Row(_1='Alice', _2=1)]
 >>> df = spark.createDataFrame(rdd, ['name', 'age'])
 >>> df.collect()
 [Row(name='Alice', age=1)]
 
 >>> from pyspark.sql import Row
 >>> Person = Row('name', 'age')
 >>> person = rdd.map(lambda r: Person(*r))
 >>> df2 = spark.createDataFrame(person)
 >>> df2.collect()
 [Row(name='Alice', age=1)]
 
 >>> from pyspark.sql.types import *
 >>> schema = StructType([
 ... StructField("name", StringType(), True),
 ... StructField("age", IntegerType(), True)])
 >>> df3 = spark.createDataFrame(rdd, schema)
 >>> df3.collect()
 [Row(name='Alice', age=1)]
 
 >>> spark.createDataFrame(df.toPandas()).collect() # doctest: +SKIP
 [Row(name='Alice', age=1)]
 >>> spark.createDataFrame(pandas.DataFrame([[1, 2]])).collect() # doctest: +SKIP
 [Row(0=1, 1=2)]
 
 >>> spark.createDataFrame(rdd, "a: string, b: int").collect()
 [Row(a='Alice', b=1)]
 >>> rdd = rdd.map(lambda row: row[1])
 >>> spark.createDataFrame(rdd, "int").collect()
 [Row(value=1)]
 >>> spark.createDataFrame(rdd, "boolean").collect() # doctest: +IGNORE_EXCEPTION_DETAIL
 Traceback (most recent call last):
 ...
 Py4JJavaError: ...

In [0]:
# creation of data frame
data = [("xavier", 30, 1), ("lucifer", 50,2),("amanadiel", 40,3),("luke", 20,4),("raneyra", 70, 5)]
col_names = ["Name", "Credits", "Rank"]
df_creds = spark.createDataFrame(data = data, schema = col_names)
df_creds.show()
print(df_creds.printSchema())

+---------+-------+----+
 Name|Credits|Rank|
+---------+-------+----+
 xavier| 30| 1|
 lucifer| 50| 2|
amanadiel| 40| 3|
 luke| 20| 4|
 raneyra| 70| 5|
+---------+-------+----+

root
-- Name: string (nullable = true)
-- Credits: long (nullable = true)
-- Rank: long (nullable = true)

None

In [0]:
from pyspark.sql.types import *
# to change the datatypes for the data frames
col_schema =StructType([StructField (name = "Name",dataType = StringType()),
            StructField (name = "Credits", dataType = IntegerType()),
            StructField (name = "Rank", dataType = IntegerType())])
df_creds = spark.createDataFrame(data = data , schema=col_schema)
df_creds.show()
df_creds.printSchema()

+---------+-------+----+
 Name|Credits|Rank|
+---------+-------+----+
 xavier| 30| 1|
 lucifer| 50| 2|
amanadiel| 40| 3|
 luke| 20| 4|
 raneyra| 70| 5|
+---------+-------+----+

root
-- Name: string (nullable = true)
-- Credits: integer (nullable = true)
-- Rank: integer (nullable = true)

In [0]:
# creation of data frame by key-value pairs
data = [{"name": "Starks", "rank":2 }, {"name":"capitan", "rank":3}, {"name":"odin_son", "rank":1}]
df_ave = spark.createDataFrame(data)
df_ave.show()
df_ave.printSchema()

+--------+----+
 name|rank|
+--------+----+
 Starks| 2|
 capitan| 3|
odin_son| 1|
+--------+----+

root
-- name: string (nullable = true)
-- rank: long (nullable = true)

In [0]:
# convertion of rdd to dataframe
data = [[1, "Alice", 25], [2, "Bob", 30]]
rdd_data = sc.parallelize(data)
print("the rdd data is ", rdd_data.collect())
schema_1 = ["id", "Name", "age"]
df_rdd = rdd_data.toDF(schema_1)
df_rdd.show()


the rdd data is [[1, 'Alice', 25], [2, 'Bob', 30]]
+---+-----+---+
 id| Name|age|
+---+-----+---+
 1|Alice| 25|
 2| Bob| 30|
+---+-----+---+

In [0]:
#reading csv files
df_csv = spark.read.format("csv").load("dbfs:/FileStore/breast_cancer_wisconsin_data.csv", header = True)
display(df_csv)

id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
842302,M,17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189
842517,M,20.57,17.77,132.9,1326,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.0186,0.0134,0.01389,0.003532,24.99,23.41,158.8,1956,0.1238,0.1866,0.2416,0.186,0.275,0.08902
84300903,M,19.69,21.25,130,1203,0.1096,0.1599,0.1974,0.1279,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.00615,0.04006,0.03832,0.02058,0.0225,0.004571,23.57,25.53,152.5,1709,0.1444,0.4245,0.4504,0.243,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.1425,0.2839,0.2414,0.1052,0.2597,0.09744,0.4956,1.156,3.445,27.23,0.00911,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.5,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.173
84358402,M,20.29,14.34,135.1,1297,0.1003,0.1328,0.198,0.1043,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.01149,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.2,1575,0.1374,0.205,0.4,0.1625,0.2364,0.07678
843786,M,12.45,15.7,82.57,477.1,0.1278,0.17,0.1578,0.08089,0.2087,0.07613,0.3345,0.8902,2.217,27.19,0.00751,0.03345,0.03672,0.01137,0.02165,0.005082,15.47,23.75,103.4,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.1244
844359,M,18.25,19.98,119.6,1040,0.09463,0.109,0.1127,0.074,0.1794,0.05742,0.4467,0.7732,3.18,53.91,0.004314,0.01382,0.02254,0.01039,0.01369,0.002179,22.88,27.66,153.2,1606,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368
84458202,M,13.71,20.83,90.2,577.9,0.1189,0.1645,0.09366,0.05985,0.2196,0.07451,0.5835,1.377,3.856,50.96,0.008805,0.03029,0.02488,0.01448,0.01486,0.005412,17.06,28.14,110.6,897,0.1654,0.3682,0.2678,0.1556,0.3196,0.1151
844981,M,13,21.82,87.5,519.8,0.1273,0.1932,0.1859,0.09353,0.235,0.07389,0.3063,1.002,2.406,24.32,0.005731,0.03502,0.03553,0.01226,0.02143,0.003749,15.49,30.73,106.2,739.3,0.1703,0.5401,0.539,0.206,0.4378,0.1072
84501001,M,12.46,24.04,83.97,475.9,0.1186,0.2396,0.2273,0.08543,0.203,0.08243,0.2976,1.599,2.039,23.94,0.007149,0.07217,0.07743,0.01432,0.01789,0.01008,15.09,40.68,97.65,711.4,0.1853,1.058,1.105,0.221,0.4366,0.2075


In [0]:
data = [("xavier", 30, 1), ("lucifer", 50,2),("amanadiel", 40,3),("luke", 20,4),("raneyra", 70, 5)]
col_names = ["Name", "Credits", "Rank"]
df_creds = spark.createDataFrame(data = data, schema = col_names)
df_creds.show()
print(df_creds.printSchema())

+---------+-------+----+
 Name|Credits|Rank|
+---------+-------+----+
 xavier| 30| 1|
 lucifer| 50| 2|
amanadiel| 40| 3|
 luke| 20| 4|
 raneyra| 70| 5|
+---------+-------+----+

root
-- Name: string (nullable = true)
-- Credits: long (nullable = true)
-- Rank: long (nullable = true)

None

In [0]:
df_creds.write.options(header = True).csv("dbfs:/FileStore/df_creds")

In [0]:
df = spark.read.format('csv').load("dbfs:/FileStore/df_creds", header = True)
df.show()

+---------+-------+----+
 Name|Credits|Rank|
+---------+-------+----+
 luke| 20| 4|
 raneyra| 70| 5|
amanadiel| 40| 3|
 lucifer| 50| 2|
 xavier| 30| 1|
+---------+-------+----+

In [0]:
data = [("xavier", 30, 1), ("lucifer", 50,2),("amanadiel", 40,3),("luke", 20,4),("raneyra", 70, 5)]
col_names = ["Name", "Credits", "Rank"]
df_creds = spark.createDataFrame(data = data, schema = col_names)
df_creds.show()
print(df_creds.printSchema())

+---------+-------+----+
 Name|Credits|Rank|
+---------+-------+----+
 xavier| 30| 1|
 lucifer| 50| 2|
amanadiel| 40| 3|
 luke| 20| 4|
 raneyra| 70| 5|
+---------+-------+----+

root
-- Name: string (nullable = true)
-- Credits: long (nullable = true)
-- Rank: long (nullable = true)

None

In [0]:
df_creds = df_creds.withColumnRenamed("Credits", "Points")
df_creds.show()

+---------+------+----+
 Name|Points|Rank|
+---------+------+----+
 xavier| 30| 1|
 lucifer| 50| 2|
amanadiel| 40| 3|
 luke| 20| 4|
 raneyra| 70| 5|
+---------+------+----+

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
data_got = [(1, ("Dani", "Targe"), 500), (2, ("Jhon","snow"),400), (3,("Tyrion", "lani"),300), (4,("Arya", "Stark"),400)]
full_name = StructType([StructField(name= "First Name", dataType = StringType()),StructField(name="Last Name", dataType=StringType())])
schema = StructType([StructField (name= "S.no", dataType=IntegerType()),StructField(name="name",dataType=full_name),StructField(name="Credits",dataType=IntegerType())])
df_got = spark.createDataFrame(data_got, schema)

In [0]:
display(df_got)
print(df_got.printSchema)

S.no,name,Credits
1,"List(Dani, Targe)",500
2,"List(Jhon, snow)",400
3,"List(Tyrion, lani)",300
4,"List(Arya, Stark)",400


<bound method DataFrame.printSchema of DataFrame[S.no: int, name: struct<First Name:string,Last Name:string>, Credits: int]>

In [0]:
#Array
from pyspark.sql.types import ArrayType
data_arr = [("abc",[1,2,3]),("def",[4,5,6]),("ghi",[7,8,9])]
schema = StructType([StructField(name="id", dataType=StringType()),StructField (name="numbers",dataType=ArrayType(IntegerType()))])
df_arr = spark.createDataFrame(data_arr, schema)
df_arr.show()
print(df_arr.printSchema)

+---+---------+
 id| numbers|
+---+---------+
abc|[1, 2, 3]|
def|[4, 5, 6]|
ghi|[7, 8, 9]|
+---+---------+

<bound method DataFrame.printSchema of DataFrame[id: string, numbers: array<int>]>

In [0]:
help(df_arr.withColumn)

Help on method withColumn in module pyspark.sql.dataframe:

withColumn(colName, col) method of pyspark.sql.dataframe.DataFrame instance
 Returns a new :class:`DataFrame` by adding a column or replacing the
 existing column that has the same name.
 
 The column expression must be an expression over this :class:`DataFrame`; attempting to add
 a column from some other :class:`DataFrame` will raise an error.
 
 .. versionadded:: 1.3.0
 
 Parameters
 ----------
 colName : str
 string, name of the new column.
 col : :class:`Column`
 a :class:`Column` expression for the new column.
 
 Notes
 -----
 This method introduces a projection internally. Therefore, calling it multiple
 times, for instance, via loops in order to add multiple columns can generate big
 plans which can cause performance issues and even `StackOverflowException`.
 To avoid this, use :func:`select` with the multiple columns at once.
 
 Examples
 --------
 >>> df.withColumn('age2', df.age + 2).collect()
 [Row(age=2, name='Alice', age2=4), Row(age=5, name='Bob', age2=7)]

In [0]:
from pyspark.sql.functions import col,array
df_arr2 = df_arr.withColumn("first number", col("numbers")[0])
df_arr2 = df_arr2.withColumn("second number", df_arr2.numbers[1])
df_arr2.show()

+---+---------+------------+-------------+
 id| numbers|first number|second number|
+---+---------+------------+-------------+
abc|[1, 2, 3]| 1| 2|
def|[4, 5, 6]| 4| 5|
ghi|[7, 8, 9]| 7| 8|
+---+---------+------------+-------------+

In [0]:
from pyspark.sql.functions import *
df_arr_explode = df_arr.withColumn("explode column", explode(col('numbers'))).select ("id", "explode column")
df_arr_explode.show()

+---+--------------+
 id|explode column|
+---+--------------+
abc| 1|
abc| 2|
abc| 3|
def| 4|
def| 5|
def| 6|
ghi| 7|
ghi| 8|
ghi| 9|
+---+--------------+

In [0]:
data_details = [("shan", "spark, databricks, python"), ("cahill","python, scala, java"), ("glassman", "html, css, bootstrap")]
col_names = ['name', 'skills']
df_details = spark.createDataFrame(data_details, col_names)
df_details = df_details.withColumn("skills array", split(col('skills'), ',' ))
display(df_details)

name,skills,skills array
shan,"spark, databricks, python","List(spark, databricks, python)"
cahill,"python, scala, java","List(python, scala, java)"
glassman,"html, css, bootstrap","List(html, css, bootstrap)"


In [0]:
df_sample = df_details.withColumn("name skills", array(col('name'), col('skills'))).select('name skills')
df_sample.show(truncate=False)

+---------------------------------+
name skills |
+---------------------------------+
[shan, spark, databricks, python]|
[cahill, python, scala, java] |
[glassman, html, css, bootstrap] |
+---------------------------------+

In [0]:
df_sample_arr_con = df_details.withColumn('arr_cont', array_contains(col('skills array'), " java"))
df_sample_arr_con.show(truncate = False)

+--------+-------------------------+-----------------------------+--------+
name |skills |skills array |arr_cont|
+--------+-------------------------+-----------------------------+--------+
shan |spark, databricks, python|[spark, databricks, python]|false |
cahill |python, scala, java |[python, scala, java] |true |
glassman|html, css, bootstrap |[html, css, bootstrap] |false |
+--------+-------------------------+-----------------------------+--------+

In [0]:
data_map = [("dani", {'hair':'white', 'eye':'blue'}),("jhon", {'hair':'black', 'eye':'brown'}), ('arya',{'hair':'black', 'eye':'red'})]
col_names_map = ['name', 'properties']
df_map = spark.createDataFrame(data_map, col_names_map)
display(df_map)
df_map.printSchema()

name,properties
dani,"Map(eye -> blue, hair -> white)"
jhon,"Map(eye -> brown, hair -> black)"
arya,"Map(eye -> red, hair -> black)"


root
-- name: string (nullable = true)
-- properties: map (nullable = true)
 |-- key: string
 |-- value: string (valueContainsNull = true)

In [0]:
df_hair = df_map.withColumn('hair', df_map.properties['hair']).select('name','hair')
df_hair.show()

+----+-----+
name| hair|
+----+-----+
dani|white|
jhon|black|
arya|black|
+----+-----+

In [0]:
df_map_explode = df_map.select('name', 'properties', explode(df_map.properties))
df_map_explode.show(truncate=False)

+----+-----------------------------+----+-----+
name|properties |key |value|
+----+-----------------------------+----+-----+
dani|{eye -> blue, hair -> white} |eye |blue |
dani|{eye -> blue, hair -> white} |hair|white|
jhon|{eye -> brown, hair -> black}|eye |brown|
jhon|{eye -> brown, hair -> black}|hair|black|
arya|{eye -> red, hair -> black} |eye |red |
arya|{eye -> red, hair -> black} |hair|black|
+----+-----------------------------+----+-----+

In [0]:
df_map_keys = df_map.select('name',map_keys(df_map.properties))
df_map_keys.show()

+----+--------------------+
name|map_keys(properties)|
+----+--------------------+
dani| [eye, hair]|
jhon| [eye, hair]|
arya| [eye, hair]|
+----+--------------------+

In [0]:
df_map_values = df_map.select('name', map_values(df_map.properties))
df_map_values.show()

+----+----------------------+
name|map_values(properties)|
+----+----------------------+
dani| [blue, white]|
jhon| [brown, black]|
arya| [red, black]|
+----+----------------------+

In [0]:
from pyspark.sql import Row
help(Row)

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 Row(*args, **kwargs)
 
 A row in :class:`DataFrame`.
 The fields in it can be accessed:
 
 * like attributes (``row.key``)
 * like dictionary values (``row[key]``)
 
 ``key in row`` will search through row keys.
 
 Row can be used to create a row object by using named arguments.
 It is not allowed to omit a named argument to represent that the value is
 None or missing. This should be explicitly set to None in this case.
 
 .. versionchanged:: 3.0.0
 Rows created from named arguments no longer have
 field names sorted alphabetically and will be ordered in the position as
 entered.
 
 Examples
 --------
 >>> row = Row(name="Alice", age=11)
 >>> row
 Row(name='Alice', age=11)
 >>> row['name'], row['age']
 ('Alice', 11)
 >>> row.name, row.age
 ('Alice', 11)
 >>> 'name' in row
 True
 >>> 'wrong_key' in row
 False
 
 Row also can be used to create another Row like class, then it
 could be used to create Row objects, such as
 
 >>> Person = Row("name", "age")
 >>> Person
 <Row('name', 'age')>
 >>> 'name' in Person
 True
 >>> 'wrong_key' in Person
 False
 >>> Person("Alice", 11)
 Row(name='Alice', age=11)
 
 This form can also be used to create rows as tuple values, i.e. with unnamed
 fields.
 
 >>> row1 = Row("Alice", 11)
 >>> row2 = Row(name="Alice", age=11)
 >>> row1 == row2
 True
 
 Method resolution order:
 Row
 builtins.tuple
 builtins.object
 
 Methods defined here:
 
 __call__(self, *args)
 create new Row object
 
 __contains__(self, item)
 Return key in self.
 
 __getattr__(self, item)
 
 __getitem__(self, item)
 Return self[key].
 
 __reduce__(self)
 Returns a tuple so Python knows how to pickle Row.
 
 __repr__(self)
 Printable representation of Row used in Python REPL.
 
 __setattr__(self, key, value)
 Implement setattr(self, name, value).
 
 asDict(self, recursive=False)
 Return as a dict
 
 Parameters
 ----------
 recursive : bool, optional
 turns the nested Rows to dict (default: False).
 
 Notes
 -----
 If a row contains duplicate field names, e.g., the rows of a join
 between two :class:`DataFrame` that both have the fields of same names,
 one of the duplicate fields will be selected by ``asDict``. ``__getitem__``
 will also return one of the duplicate fields, however returned value might
 be different to ``asDict``.
 
 Examples
 --------
 >>> Row(name="Alice", age=11).asDict() == {'name': 'Alice', 'age': 11}
 True
 >>> row = Row(key=1, value=Row(name='a', age=2))
 >>> row.asDict() == {'key': 1, 'value': Row(name='a', age=2)}
 True
 >>> row.asDict(True) == {'key': 1, 'value': {'name': 'a', 'age': 2}}
 True
 
 ----------------------------------------------------------------------
 Static methods defined here:
 
 __new__(cls, *args, **kwargs)
 Create and return a new object. See help(type) for accurate signature.
 
 ----------------------------------------------------------------------
 Data descriptors defined here:
 
 __dict__
 dictionary for instance variables (if defined)
 
 ----------------------------------------------------------------------
 Methods inherited from builtins.tuple:
 
 __add__(self, value, /)
 Return self+value.
 
 __eq__(self, value, /)
 Return self==value.
 
 __ge__(self, value, /)
 Return self>=value.
 
 __getattribute__(self, name, /)
 Return getattr(self, name).
 
 __getnewargs__(self, /)
 
 __gt__(self, value, /)
 Return self>value.
 
 __hash__(self, /)
 Return hash(self).
 
 __iter__(self, /)
 Implement iter(self).
 
 __le__(self, value, /)
 Return self<=value.
 
 __len__(self, /)
 Return len(self).
 
 __lt__(self, value, /)
 Return self<value.
 
 __mul__(self, value, /)
 Return self*value.
 
 __ne__(self, value, /)
 Return self!=value.
 
 __rmul__(self, value, /)
 Return value*self.
 
 count(self, value, /)
 Return number of occurrences of value.
 
 index(self, value, start=0, stop=9223372036854775807, /)
 Return first index of value.
 
 Raises ValueError if the value is not present.

In [0]:
row1 = Row(name = "Tim", salary= 50000)
row2 = Row(name = "Lucy", salary= 45000)
data_0 = [row1, row2]
df_row_0 = spark.createDataFrame(data_0)
person = Row('name','salary')
row3 = person('nolan', 45000)
row4 = person('alex', 40000)
data_1 = [row3, row4]
df_row_1 = spark.createDataFrame(data_1)
df_row = df_row_0.union(df_row_1)
display(df_row)


name,salary
Tim,50000
Lucy,45000
nolan,45000
alex,40000


In [0]:
# nested struct type
data = [Row(name = "Buck", prop = Row(hair = 'black', eye= 'black')),Row(name = "steve", prop = Row(hair = 'Blonde', eye = 'Brown'))]
df_nested = spark.createDataFrame(data)
display(df_nested)

name,prop
Buck,"List(black, black)"
steve,"List(Blonde, Brown)"


In [0]:
data = [("xavier", 30, 1), ("lucifer", 50,2),("amanadiel", 40,3),("luke", 20,4),("raneyra", 70, 5)]
df_dd = spark.createDataFrame(data)

In [0]:
from pyspark.sql.functions import lit
df_dd = df_dd.withColumn("new_Col", lit("new_col_values"))
display(df_dd)

_1,_2,_3,new_Col
xavier,30,1,new_col_values
lucifer,50,2,new_col_values
amanadiel,40,3,new_col_values
luke,20,4,new_col_values
raneyra,70,5,new_col_values


In [0]:
data = [("xavier", 30, 1, 'm'), ("lucifer", 50,2,'m'),("amani", 40,3,'f'),("luke", 20,4,' '),("raneyra", 70, 5,'f')]
col_name = ['name','id', 'points', 'gender']
df_data = spark.createDataFrame(data,col_name)
df_data.display()

name,id,points,gender
xavier,30,1,m
lucifer,50,2,m
amani,40,3,f
luke,20,4,
raneyra,70,5,f


In [0]:
from pyspark.sql.functions import when
help(when)

Help on function when in module pyspark.sql.functions:

when(condition, value)
 Evaluates a list of conditions and returns one of multiple possible result expressions.
 If :func:`pyspark.sql.Column.otherwise` is not invoked, None is returned for unmatched
 conditions.
 
 .. versionadded:: 1.4.0
 
 Parameters
 ----------
 condition : :class:`~pyspark.sql.Column`
 a boolean :class:`~pyspark.sql.Column` expression.
 value :
 a literal value, or a :class:`~pyspark.sql.Column` expression.
 
 >>> df.select(when(df['age'] == 2, 3).otherwise(4).alias("age")).collect()
 [Row(age=3), Row(age=4)]
 
 >>> df.select(when(df.age == 2, df.age + 1).alias("age")).collect()
 [Row(age=3), Row(age=None)]

In [0]:
df_data_1 = df_data.select(
    df_data.name, 
    df_data.id,
    df_data.points, 
    when(df_data.gender=='m','male')
    .when(df_data.gender=='f','female')
    .otherwise('unknown').alias('gender'))
display(df_data_1)

name,id,points,gender
xavier,30,1,male
lucifer,50,2,male
amani,40,3,female
luke,20,4,unknown
raneyra,70,5,female


In [0]:
df_data_2 = df_data.select(df_data.name, df_data.id.alias('emp_id'))
display(df_data_2)

name,emp_id
xavier,30
lucifer,50
amani,40
luke,20
raneyra,70


In [0]:
from pyspark.sql.functions import desc, asc
df_data_3 = df_data.sort(df_data.id.desc())
display(df_data_3)

name,id,points,gender
raneyra,70,5,f
lucifer,50,2,m
amani,40,3,f
xavier,30,1,m
luke,20,4,


In [0]:
df_data_3 = df_data_3.sort(df_data_3.id.asc())
df_data_3.display(df)

name,id,points,gender
luke,20,4,
xavier,30,1,m
amani,40,3,f
lucifer,50,2,m
raneyra,70,5,f


In [0]:
df_data_3.printSchema()

root
-- name: string (nullable = true)
-- id: long (nullable = true)
-- points: long (nullable = true)
-- gender: string (nullable = true)

In [0]:
from pyspark.sql.types import IntegerType
df_data_3 = df_data_3.select(df_data_3.id.cast(IntegerType()))
df_data_3.printSchema()

root
-- id: integer (nullable = true)

In [0]:
df_data.filter(df_data.name.like("%e%")).show()

+-------+---+------+------+
 name| id|points|gender|
+-------+---+------+------+
 xavier| 30| 1| m|
lucifer| 50| 2| m|
 luke| 20| 4| |
raneyra| 70| 5| f|
+-------+---+------+------+

In [0]:
data = [("xavier", 30, 1, 'male'), ("lucifer", 50,2,'male'),("amani", 40,3,'female'),("luke", 20,4,'male'),("raneyra", 70, 5,'female')]
col_name = ['name','points', 'id', 'gender']
df_data = spark.createDataFrame(data,col_name)
df_data.display()

name,points,id,gender
xavier,30,1,male
lucifer,50,2,male
amani,40,3,female
luke,20,4,male
raneyra,70,5,female


In [0]:
df_data.filter(df_data.gender == 'male').show()

+-------+------+---+------+
 name|points| id|gender|
+-------+------+---+------+
 xavier| 30| 1| male|
lucifer| 50| 2| male|
 luke| 20| 4| male|
+-------+------+---+------+

In [0]:
df_data.where((df_data.gender=='female')&(df_data.points > 40)).show()

+-------+------+---+------+
 name|points| id|gender|
+-------+------+---+------+
raneyra| 70| 5|female|
+-------+------+---+------+

In [0]:
df_data.dropDuplicates(['gender']).show()

+------+------+---+------+
 name|points| id|gender|
+------+------+---+------+
 amani| 40| 3|female|
xavier| 30| 1| male|
+------+------+---+------+

In [0]:
df_data.sort(df_data.name.asc()).show()

+-------+------+---+------+
 name|points| id|gender|
+-------+------+---+------+
 amani| 40| 3|female|
lucifer| 50| 2| male|
 luke| 20| 4| male|
raneyra| 70| 5|female|
 xavier| 30| 1| male|
+-------+------+---+------+

In [0]:
df_data.orderBy(df_data.name.asc()).show()

+-------+------+---+------+
 name|points| id|gender|
+-------+------+---+------+
 amani| 40| 3|female|
lucifer| 50| 2| male|
 luke| 20| 4| male|
raneyra| 70| 5|female|
 xavier| 30| 1| male|
+-------+------+---+------+

In [0]:
data = [{"name": "Starks", "rank":2 }, {"name":"capitan", "rank":3}, {"name":"odin_son", "rank":1}]
df_1 = spark.createDataFrame(data)
df_2= spark.createDataFrame(data)

In [0]:
df_1.union(df_2).show()
df_1.unionAll(df_2).show()

+--------+----+
 name|rank|
+--------+----+
 Starks| 2|
 capitan| 3|
odin_son| 1|
 Starks| 2|
 capitan| 3|
odin_son| 1|
+--------+----+

+--------+----+
 name|rank|
+--------+----+
 Starks| 2|
 capitan| 3|
odin_son| 1|
 Starks| 2|
 capitan| 3|
odin_son| 1|
+--------+----+

In [0]:
data_2 = [("xavier", 30, 1, 'male','man'), ("lucifer", 50,2,'male','man'),("amani", 40,3,'female','dragon'),("luke", 20,4,'male','dragon'),("raneyra", 70, 5,'female','dragon'),('rhynees',55,6,'female','dragon'),('helena',35,7,'female','man')]
col_name = ['name','points', 'id', 'gender','controller']
df_data_2 = spark.createDataFrame(data_2,col_name)

In [0]:
df_data_2.groupBy('gender','controller').count().show()

+------+----------+-----+
gender|controller|count|
+------+----------+-----+
 male| man| 2|
female| dragon| 3|
 male| dragon| 1|
female| man| 1|
+------+----------+-----+

In [0]:
df_data_2.groupBy('gender').min('points').show()

+------+-----------+
gender|min(points)|
+------+-----------+
 male| 20|
female| 35|
+------+-----------+

In [0]:
from pyspark.sql.functions import count, min, max
df_data_2.groupBy('controller').agg(count('*') .alias('total number'),min('points').alias('min points'),max('id').alias('max id')).show()

+----------+------------+----------+------+
controller|total number|min points|max id|
+----------+------------+----------+------+
 man| 3| 30| 7|
 dragon| 4| 20| 6|
+----------+------------+----------+------+

In [0]:
#joins
data1 = [(1 ,'tully', 28, 'male'),(2, 'vaseryas', 69, 'male'), (5,'riyan',24,'female'),(6,'corlyn',38,'female')]
schema1 = ['id','name','age','gender']
data2 = [(1,'master'),(2,'warrior'),(4,'warrior'),(5,'master')]
schema2 = ['no', 'profession']
df1 = spark.createDataFrame(data1,schema1)
df2 = spark.createDataFrame(data2,schema2)

In [0]:
ij = df1.join(df2, df1.id == df2.no,'inner')
lj = df1.join(df2, df1.id == df2.no,'left')
rj = df1.join(df2, df1.id == df2.no,'right')
fj = df1.join(df2, df1.id == df2.no,'full') 
ij.show()
lj.show()
rj.show()
fj.show()

+---+--------+---+------+---+----------+
 id| name|age|gender| no|profession|
+---+--------+---+------+---+----------+
 1| tully| 28| male| 1| master|
 2|vaseryas| 69| male| 2| warrior|
 5| riyan| 24|female| 5| master|
+---+--------+---+------+---+----------+

+---+--------+---+------+----+----------+
 id| name|age|gender| no|profession|
+---+--------+---+------+----+----------+
 1| tully| 28| male| 1| master|
 2|vaseryas| 69| male| 2| warrior|
 5| riyan| 24|female| 5| master|
 6| corlyn| 38|female|null| null|
+---+--------+---+------+----+----------+

+----+--------+----+------+---+----------+
 id| name| age|gender| no|profession|
+----+--------+----+------+---+----------+
 1| tully| 28| male| 1| master|
 2|vaseryas| 69| male| 2| warrior|
null| null|null| null| 4| warrior|
 5| riyan| 24|female| 5| master|
+----+--------+----+------+---+----------+

+----+--------+----+------+----+----------+
 id| name| age|gender| no|profession|
+----+--------+----+------+----+----------+
 1| tully| 28| male| 1| master|
 2|vaseryas| 69| male| 2| warrior|
null| null|null| null| 4| warrior|
 5| riyan| 24|female| 5| master|
 6| corlyn| 38|female|null| null|
+----+--------+----+------+----+----------+

In [0]:
lsj = df1.join(df2, df1.id == df2.no, 'leftsemi')
lsj.show()
laj  = df1.join(df2, df1.id == df2.no, 'leftanti')
laj.show()

+---+--------+---+------+
 id| name|age|gender|
+---+--------+---+------+
 1| tully| 28| male|
 2|vaseryas| 69| male|
 5| riyan| 24|female|
+---+--------+---+------+

+---+------+---+------+
 id| name|age|gender|
+---+------+---+------+
 6|corlyn| 38|female|
+---+------+---+------+

In [0]:
data = [(1 ,'tully', 28, 'male','maester'),(2, 'vaseryas', 69, 'male','king'), (3,'riyan',24,'female','warrior'),(4,'corlyni',38,'female','master'),(5, 'harwin',30,'male','warrior'), (6,'helena', 24,'female', 'warrior')]
col_name = ['id','name','age','gender','designation']
df = spark.createDataFrame(data,col_name)
df.display()

id,name,age,gender,designation
1,tully,28,male,maester
2,vaseryas,69,male,king
3,riyan,24,female,warrior
4,corlyni,38,female,master
5,harwin,30,male,warrior
6,helena,24,female,warrior


In [0]:
df_sam = df.groupBy('designation').pivot('gender').count()
df_sam.display()

designation,female,male
maester,null,1
master,1,null
king,null,1
warrior,2,1


In [0]:
data = [("John", None), ("Jane", "female"), ("Doe", "male")]
columns = ["Name", "Gender"]
df_sam = spark.createDataFrame(data, schema=columns)

# Correct usage of fillna
df_filled = df_sam.fillna('nope',['Gender'])
display(df_filled)

Name,Gender
John,nope
Jane,female
Doe,male


In [0]:
#sample
df= spark.range(start =1, end = 1001)
df1 = df.sample(fraction=0.4, seed=20)
df2 = df.sample(fraction = 0.1, seed = 10)  

In [0]:
df1.display()

id
4
5
7
9
10
11
14
15
16
21


In [0]:
df2.display()

id
21
23
26
27
43
45
48
52
58
68


In [0]:
df_01 = df_sam.collect()
print(df_01)

[Row(Name='John', Gender=None), Row(Name='Jane', Gender='female'), Row(Name='Doe', Gender='male')]

In [0]:
print(df_01[1])
print(df_01[0][0])
print(df_01[0:2][0:2])

Row(Name='Jane', Gender='female')
John
[Row(Name='John', Gender=None), Row(Name='Jane', Gender='female')]

In [0]:
#transform
data = [(1 ,'tully', 28, 'male','maester'),(2, 'vaseryas', 69, 'male','king'), (3,'riyan',24,'female','warrior'),(4,'corlyni',38,'female','master'),(5, 'harwin',30,'male','warrior'), (6,'helena', 24,'female', 'warrior')]
col_name = ['id','name','age','gender','designation']
df = spark.createDataFrame(data,col_name)
df.display()

id,name,age,gender,designation
1,tully,28,male,maester
2,vaseryas,69,male,king
3,riyan,24,female,warrior
4,corlyni,38,female,master
5,harwin,30,male,warrior
6,helena,24,female,warrior


In [0]:
from pyspark.sql.functions import upper
def convertToUpper(df):
    return df.withColumn("name",upper(df['name']))
def double_the_id(df):
    return df.withColumn('id',df.id*2)
df1 = df.transform(convertToUpper).transform(double_the_id)

df1.display()

id,name,age,gender,designation
2,TULLY,28,male,maester
4,VASERYAS,69,male,king
6,RIYAN,24,female,warrior
8,CORLYNI,38,female,master
10,HARWIN,30,male,warrior
12,HELENA,24,female,warrior


In [0]:
data2 = [(1,'alex',['javasript','html']),(2,'brownie',['css','booststrap']),(3,'casy',['python','java'])]
col_namee=['id','name','skills']
df2 = spark.createDataFrame(data2,col_namee)
df2.display()

id,name,skills
1,alex,"List(javasript, html)"
2,brownie,"List(css, booststrap)"
3,casy,"List(python, java)"


In [0]:
from pyspark.sql.functions import transform
df2.select('id','name',transform('skills', lambda x:upper(x)).alias('skills')).show()

+---+-------+-----------------+
 id| name| skills|
+---+-------+-----------------+
 1| alex|[JAVASRIPT, HTML]|
 2|brownie|[CSS, BOOSTSTRAP]|
 3| casy| [PYTHON, JAVA]|
+---+-------+-----------------+

In [0]:
df.createOrReplaceTempView('details')

In [0]:
%sql
select id, upper(name), designation, age from details where age>=30 order by designation asc

id,upper(name),designation,age
2,VASERYAS,king,69
4,CORLYNI,master,38
5,HARWIN,warrior,30


In [0]:
data3 = [(1,'chole',300,30000),(2,'chloie',400,40000),(3,'casy',550,55000)]
colName = ["s.no",'name','credits','rewards']
df3 = spark.createDataFrame(data3,colName)
df3.display()

s.no,name,credits,rewards
1,chole,300,30000
2,chloie,400,40000
3,casy,550,55000


In [0]:
def total(c,r):
    return c+r
from pyspark.sql.functions import udf,col
from pyspark.sql.types import IntegerType


In [0]:
total_recived = udf(lambda x,y:total(x,y),IntegerType())
df3.select('*',total_recived(col('credits'),col('rewards')).alias('total_amount')).show()

+----+------+-------+-------+------------+
s.no| name|credits|rewards|total_amount|
+----+------+-------+-------+------------+
 1| chole| 300| 30000| 30300|
 2|chloie| 400| 40000| 40400|
 3| casy| 550| 55000| 55550|
+----+------+-------+-------+------------+

In [0]:
from pyspark.sql.functions import current_date, to_date, lit, date_format

In [0]:
df = spark.range(1)
df.withColumn('todays date', current_date()).show()

+---+-----------+
 id|todays date|
+---+-----------+
 0| 2024-07-03|
+---+-----------+

In [0]:
#date_Format converts datetype to specified format
df.withColumn('newFormat', date_format(lit(current_date()),'dd.MM.yyyy')).show()

+---+----------+
 id| newFormat|
+---+----------+
 0|03.07.2024|
+---+----------+

In [0]:
#to_date: converts string values of date to datetype
df.withColumn('newdatecol',to_date(lit('25-12-2023'),'dd-MM-yyyy')).show()

+---+----------+
 id|newdatecol|
+---+----------+
 0|2023-12-25|
+---+----------+

In [0]:
from pyspark.sql.functions import datediff, months_between, add_months, date_add,year,month

In [0]:
date = [('2024-02-17','2023-02-17'),('2024-01-31','2023-07-12'),('2024-02-25','2023-04-26'),('2024-07-17','2023-08-25'),('2024-05-15','2023-07-19')]
col_name = ['date1','date2']
df_date = spark.createDataFrame(date, col_name)
df_date.display()

date1,date2
2024-02-17,2023-02-17
2024-01-31,2023-07-12
2024-02-25,2023-04-26
2024-07-17,2023-08-25
2024-05-15,2023-07-19


In [0]:
df_date.withColumn('dateDiff', datediff(df_date.date1, df_date.date2)).display()

date1,date2,dateDiff
2024-02-17,2023-02-17,365
2024-01-31,2023-07-12,203
2024-02-25,2023-04-26,305
2024-07-17,2023-08-25,327
2024-05-15,2023-07-19,301


In [0]:
df_date.withColumn('months between', months_between(df_date.date1,df_date.date2)).display()

date1,date2,months between
2024-02-17,2023-02-17,12.0
2024-01-31,2023-07-12,6.61290323
2024-02-25,2023-04-26,9.96774194
2024-07-17,2023-08-25,10.74193548
2024-05-15,2023-07-19,9.87096774


In [0]:
df_date.withColumn('added months', add_months(df_date.date2,4)).display()

date1,date2,added months
2024-02-17,2023-02-17,2023-06-17
2024-01-31,2023-07-12,2023-11-12
2024-02-25,2023-04-26,2023-08-26
2024-07-17,2023-08-25,2023-12-25
2024-05-15,2023-07-19,2023-11-19


In [0]:
df_date.withColumn('monthdifference', add_months(df_date.date1,-5)).display()

date1,date2,monthdifference
2024-02-17,2023-02-17,2023-09-17
2024-01-31,2023-07-12,2023-08-31
2024-02-25,2023-04-26,2023-09-25
2024-07-17,2023-08-25,2024-02-17
2024-05-15,2023-07-19,2023-12-15


In [0]:
df_date.withColumn('add date', date_add(df_date.date2,21)).display()

date1,date2,add date
2024-02-17,2023-02-17,2023-03-10
2024-01-31,2023-07-12,2023-08-02
2024-02-25,2023-04-26,2023-05-17
2024-07-17,2023-08-25,2023-09-15
2024-05-15,2023-07-19,2023-08-09


In [0]:
df_date.withColumn('sub date', date_add(df_date.date1,-29)).display()

date1,date2,sub date
2024-02-17,2023-02-17,2024-01-19
2024-01-31,2023-07-12,2024-01-02
2024-02-25,2023-04-26,2024-01-27
2024-07-17,2023-08-25,2024-06-18
2024-05-15,2023-07-19,2024-04-16


In [0]:
df_date.withColumn('year', year(df_date.date2)).show()

+----------+----------+----+
 date1| date2|year|
+----------+----------+----+
2024-02-17|2023-02-17|2023|
2024-01-31|2023-07-12|2023|
2024-02-25|2023-04-26|2023|
2024-07-17|2023-08-25|2023|
2024-05-15|2023-07-19|2023|
+----------+----------+----+

In [0]:
df_date.withColumn('month', month(df_date.date1)).display()

date1,date2,month
2024-02-17,2023-02-17,2
2024-01-31,2023-07-12,1
2024-02-25,2023-04-26,2
2024-07-17,2023-08-25,7
2024-05-15,2023-07-19,5


In [0]:
from pyspark.sql.functions import current_timestamp, to_timestamp, hour, minute, second

In [0]:
df = spark.range(1)
df1 = df.withColumn('timestamp', current_timestamp())
df1.show(truncate= False)
df1.printSchema()

+---+-----------------------+
id |timestamp |
+---+-----------------------+
0 |2024-07-03 01:19:39.094|
+---+-----------------------+

root
-- id: long (nullable = false)
-- timestamp: timestamp (nullable = false)

In [0]:
df2 = df.withColumn('to_timeStamp', to_timestamp(lit('12.25.2023 10.06.34'), 'dd.MM.yyyy HH.mm.ss'))
df2.show(truncate=False)
df2.printSchema()

+---+------------+
id |to_timeStamp|
+---+------------+
0 |null |
+---+------------+

root
-- id: long (nullable = false)
-- to_timeStamp: timestamp (nullable = true)

In [0]:
df2.select('id',current_timestamp() ,hour(current_timestamp()),minute(current_timestamp()),second(current_timestamp())).display()

id,current_timestamp(),hour(current_timestamp()),minute(current_timestamp()),second(current_timestamp())
0,2024-07-03T01:24:06.888+0000,1,24,6


In [0]:
from pyspark.sql.functions import approx_count_distinct,avg, collect_list, collect_set, countDistinct,count,row_number,rank, dense_rank
from pyspark.sql.window import Window

In [0]:
s_data = [('azej', 'Techie', 7000),('rupesh', 'manager', 10000), ('jwalkar','hr', 7000),('kira','planner',7000),('advani', 'convenor',8000),('rudra', 'data manager',1000)]
col_name = ['name','role','salary']
df = spark.createDataFrame(s_data,col_name)
df.show()
df.select(approx_count_distinct('salary')).show()
df.select(avg('salary')).show()

+-------+-------+------+
 name| role|salary|
+-------+-------+------+
 azej| Techie| 7000|
 rupesh|manager| 10000|
jwalkar| hr| 7000|
 kira|planner| 7000|
+-------+-------+------+

+-----------------------------+
approx_count_distinct(salary)|
+-----------------------------+
 2|
+-----------------------------+

+-----------+
avg(salary)|
+-----------+
 7750.0|
+-----------+

In [0]:
df.select(collect_list('salary')).show(truncate=False)
df.select(collect_set('salary')).show(truncate=False)

+-------------------------------------+
collect_list(salary) |
+-------------------------------------+
[7000, 8000, 10000, 1000, 7000, 7000]|
+-------------------------------------+

+-------------------------+
collect_set(salary) |
+-------------------------+
[8000, 1000, 7000, 10000]|
+-------------------------+

In [0]:
df.select(countDistinct('salary')).show()
df.select(count('salary')).show()

+----------------------+
count(DISTINCT salary)|
+----------------------+
 4|
+----------------------+

+-------------+
count(salary)|
+-------------+
 6|
+-------------+

In [0]:
data = [('azej', 'techie', 7000),('rupesh', 'manager', 10000), ('jwalkar','hr', 7000),('kira','techie',7000),('advani', 'hr',8000),('rudra', 'manager',1000)]
col_name = ['name','role','salary']
df = spark.createDataFrame(data,col_name)
df.show()

+-------+-------+------+
 name| role|salary|
+-------+-------+------+
 azej| techie| 7000|
 rupesh|manager| 10000|
jwalkar| hr| 7000|
 kira| techie| 7000|
 advani| hr| 8000|
 rudra|manager| 1000|
+-------+-------+------+

In [0]:
df = df.sort('role')
df.show()

+-------+-------+------+
 name| role|salary|
+-------+-------+------+
jwalkar| hr| 7000|
 advani| hr| 8000|
 rudra|manager| 1000|
 rupesh|manager| 10000|
 azej| techie| 7000|
 kira| techie| 7000|
+-------+-------+------+

In [0]:

window = Window.partitionBy('role').orderBy('salary')

In [0]:
#row_number: sequnetial row number of each window partition
df.withColumn('rowNumber', row_number().over(window)).show()

+-------+-------+------+---------+
 name| role|salary|rowNumber|
+-------+-------+------+---------+
jwalkar| hr| 7000| 1|
 advani| hr| 8000| 2|
 rudra|manager| 1000| 1|
 rupesh|manager| 10000| 2|
 azej| techie| 7000| 1|
 kira| techie| 7000| 2|
+-------+-------+------+---------+

In [0]:

#rank: provide rank to result with in a window partition with gaps
df.withColumn('rank',rank().over(window)).show()

+-------+-------+------+----+
 name| role|salary|rank|
+-------+-------+------+----+
jwalkar| hr| 7000| 1|
 advani| hr| 8000| 2|
 rudra|manager| 1000| 1|
 rupesh|manager| 10000| 2|
 azej| techie| 7000| 1|
 kira| techie| 7000| 1|
+-------+-------+------+----+

In [0]:
#dense rank : similar to rank but provides result without gaps
df.withColumn('denserank', dense_rank().over(window)).show()

+-------+-------+------+---------+
 name| role|salary|denserank|
+-------+-------+------+---------+
jwalkar| hr| 7000| 1|
 advani| hr| 8000| 2|
 rudra|manager| 1000| 1|
 rupesh|manager| 10000| 2|
 azej| techie| 7000| 1|
 kira| techie| 7000| 1|
+-------+-------+------+---------+